https://medium.com/@SSiddhant/implementing-black-scholes-model-for-option-chains-using-python-6978662bfbb6

In [1]:
from math import sqrt, log, pow, erf, e
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm

# Black-Scholes Model

In [2]:
def standard_normal_cdf(x):
    return 0.5 * (1 + erf(x / sqrt(2)))

In [3]:
def d1(S, K, t, r, vol):
    numerator = log(S/K) + (r + pow(vol, 2)/2) * t
    denominator = vol * sqrt(t)
    return numerator / denominator

In [4]:
def d2(S, K, t, r, vol):
    return d1(S, K, t, r, vol) - vol * sqrt(t)

In [5]:
def call_price(S, K, t, r, vol):
    Nd1 = standard_normal_cdf(d1(S, K, t, r, vol))
    Nd2 = standard_normal_cdf(d2(S, K, t, r, vol))
    return Nd1 * S - Nd2 * K * pow(e, -1 * r * t)

In [6]:
def put_price(S, K, t, r, vol):
    Nd1 = standard_normal_cdf(- 1 * d1(S, K, t, r, vol))
    Nd2 = standard_normal_cdf(- 1 * d2(S, K, t, r, vol))
    return Nd2 * K * pow(e, -1 * r * t) - S * Nd1

# Get data from yfinance

In [7]:
def fetch_options_data(ticker_symbol, date):
    ticker = yf.Ticker(ticker_symbol)
    options_data = ticker.option_chain(date)
    return options_data.calls, options_data.puts

In [8]:
def get_current_stock_price(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    current_price = stock.history(period='1d')['Close'].iloc[-1]
    return current_price

In [9]:
def get_10yr_treasury_rate():
    # 10 year treasury ticker symbol
    treasury_ticker = "^TNX"

    now = datetime.now()
    ten_years_ago = now.replace(year=now.year - 10)

    treasury_data = yf.download(treasury_ticker, start=ten_years_ago, end=now)
    last_yield = treasury_data['Close'].iloc[-1]
    return last_yield

# Calculate date and volatility

In [10]:
def get_date(date_str):
    # Assuming the date string is in the format 'YYYY-MM-DD'
    date_object = datetime.strptime(date_str, '%Y-%m-%d')

    current_date = datetime.now()
    years_difference = (date_object - current_date).days / 365.25
    return years_difference

In [11]:
def calculate_volatility(ticker):

    today = datetime.now()
    one_year_ago = today.replace(year=today.year-1)
    data = yf.download(ticker, start=one_year_ago, end=today)

    # Calculate daily returns
    data['Daily_Return'] = data['Adj Close'].pct_change()

    # std of daily returns
    daily_volatility = data['Daily_Return'].std()
    annualized_volatility = daily_volatility * np.sqrt(252)

    return annualized_volatility

# Calculate Greeks

In [12]:
def delta_call(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return norm.cdf(d1)

In [13]:
def gamma_call(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    numerator = norm.pdf(d1)
    denominator = S * sigma * np.sqrt(T)
    return numerator / denominator

In [14]:
def vega_call(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * norm.pdf(d1) * np.sqrt(T)

In [15]:
#https://www.geeksforgeeks.org/python-pow-function/

def theta_call(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    numerator = S * norm.pdf(d1) * sigma
    denominator = 2 * np.sqrt(T)
    return -1 * (numerator / denominator) - r * K * pow(np.exp(1), -1 * r * T) * norm.cdf(d2)

In [16]:
#https://www.geeksforgeeks.org/python-pow-function/

def rho_call(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K * T * pow(np.exp(1), -1 * r * T) * norm.cdf(d2)

In [17]:
def delta_put(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return norm.cdf(d1) - 1

In [18]:
def gamma_put(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    numerator = norm.pdf(d1)
    denominator = S * sigma * np.sqrt(T)
    return numerator / denominator

In [19]:
def vega_put(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * norm.pdf(d1) * np.sqrt(T)

In [20]:
def theta_put(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    numerator = S * norm.pdf(d1) * sigma
    denominator = 2 * np.sqrt(T)
    return -1 * (numerator / denominator) + r * K * pow(np.exp(1), -1 * r * T) * norm.cdf(-1 * d2)

In [21]:
def rho_put(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return -1 * K * T * pow(np.exp(1), -1 * r * T) * norm.cdf(-1 * d2)

# Start of the analysis

In [22]:
ticker = 'AAPL'

('2024-08-16',
 '2024-08-23',
 '2024-08-30',
 '2024-09-06',
 '2024-09-13',
 '2024-09-20',
 '2024-09-27',
 '2024-10-18',
 '2024-11-15',
 '2024-12-20',
 '2025-01-17',
 '2025-03-21',
 '2025-06-20',
 '2025-08-15',
 '2025-09-19',
 '2025-12-19',
 '2026-01-16',
 '2026-06-18',
 '2026-12-18')

In [ ]:
option = yf.Ticker(ticker)
expirations = option.options
expirations 

In [23]:
date = expirations[0]

In [24]:
calls, puts = fetch_options_data(ticker, date)
calls = pd.DataFrame(calls)
calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL240816C00005000,2024-08-05 17:41:05+00:00,5.0,206.54,0.00,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
1,AAPL240816C00015000,2024-07-16 16:16:43+00:00,15.0,218.30,0.00,0.00,0.0,0.0,6.0,0,0.000010,True,REGULAR,USD
2,AAPL240816C00050000,2024-06-13 19:47:00+00:00,50.0,164.65,179.70,181.90,0.0,0.0,10.0,15,14.179689,True,REGULAR,USD
3,AAPL240816C00080000,2024-07-11 15:06:40+00:00,80.0,147.30,134.95,137.45,0.0,0.0,2.0,84,4.883793,True,REGULAR,USD
4,AAPL240816C00085000,2024-08-09 18:42:39+00:00,85.0,130.38,0.00,0.00,0.0,0.0,10.0,0,0.000010,True,REGULAR,USD


In [25]:
S = get_current_stock_price(ticker)
r = get_10yr_treasury_rate()
t = get_date(date)
vol = calculate_volatility(ticker)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [26]:
# Converting r to percentage
r = r / 100

In [27]:
print(f"Riskfree rate: {r}")
print(f"Years difference: {t}")
print(f"The current stock price of {ticker} is: {S}")
print(f"Stock price of {ticker}: {S}")

Riskfree rate: 0.03941999912261963
Years difference: 0.008213552361396304
The current stock price of AAPL is: 216.24000549316406
Stock price of AAPL: 216.24000549316406


In [28]:
main_df = calls.copy()
columns_to_drop = ['lastTradeDate', 'lastPrice', 'volume', 'openInterest', 'contractSize', 'currency']
main_df.drop(columns=columns_to_drop, inplace=True)
main_df['bsmValuation'] = main_df.apply(lambda row: call_price(S, row['strike'], t, r, vol), axis=1)
main_df.head(10)

,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,bsmValuation
0,AAPL240816C00005000,5.0,0.00,0.00,0.0,0.0,0.000010,True,211.241624
1,AAPL240816C00015000,15.0,0.00,0.00,0.0,0.0,0.000010,True,201.244861
2,AAPL240816C00050000,50.0,179.70,181.90,0.0,0.0,14.179689,True,166.256192
3,AAPL240816C00080000,80.0,134.95,137.45,0.0,0.0,4.883793,True,136.265904
4,AAPL240816C00085000,85.0,0.00,0.00,0.0,0.0,0.000010,True,131.267522
5,AAPL240816C00090000,90.0,0.00,0.00,0.0,0.0,0.000010,True,126.269141
6,AAPL240816C00095000,95.0,0.00,0.00,0.0,0.0,0.000010,True,121.270759
7,AAPL240816C00100000,100.0,0.00,0.00,0.0,0.0,0.000010,True,116.272378
8,AAPL240816C00105000,105.0,0.00,0.00,0.0,0.0,0.000010,True,111.273997
9,AAPL240816C00110000,110.0,0.00,0.00,0.0,0.0,0.000010,True,106.275615


In [38]:
print(f"Current Spot is {S:.2f}")
print(f"Time to expiry is {t:.2f} years")
print(f"Current risk-free interest rate is {r*100:.2f}%")
print(f"Historical Volatility for the stock is {vol:.2f}")

Current Spot is 216.24
Time to expiry is 0.01 years
Current risk-free interest rate is 3.94%
Historical Volatility for the stock is 0.23


In [29]:
greeks_df = main_df.copy()
greeks_df['delta'] = greeks_df.apply(lambda row: delta_call(S, row['strike'], t, r, vol), axis=1)
greeks_df['gamma'] = greeks_df.apply(lambda row: gamma_call(S, row['strike'], t, r, vol), axis=1)
greeks_df['vega'] = greeks_df.apply(lambda row: vega_call(S, row['strike'], t, r, vol), axis=1)
greeks_df['theta'] = greeks_df.apply(lambda row: theta_call(S, row['strike'], t, r, vol), axis=1)
greeks_df['rho'] = greeks_df.apply(lambda row: rho_call(S, row['strike'], t, r, vol), axis=1)

In [31]:
greeks_df

,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,bsmValuation,delta,gamma,vega,theta,rho
0,AAPL240816C00005000,5.0,0.00,0.00,0.0,0.0,0.000010,True,211.241624,1.000000e+00,0.000000e+00,0.000000e+00,-4.099068e-02,1.141442e+00
1,AAPL240816C00015000,15.0,0.00,0.00,0.0,0.0,0.000010,True,201.244861,1.000000e+00,0.000000e+00,0.000000e+00,-1.229721e-01,3.424325e+00
2,AAPL240816C00050000,50.0,179.70,181.90,0.0,0.0,14.179689,True,166.256192,1.000000e+00,0.000000e+00,0.000000e+00,-4.099068e-01,1.141442e+01
3,AAPL240816C00080000,80.0,134.95,137.45,0.0,0.0,4.883793,True,136.265904,1.000000e+00,0.000000e+00,0.000000e+00,-6.558510e-01,1.826307e+01
4,AAPL240816C00085000,85.0,0.00,0.00,0.0,0.0,0.000010,True,131.267522,1.000000e+00,0.000000e+00,0.000000e+00,-6.968416e-01,1.940451e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,AAPL240816C00330000,330.0,0.00,0.00,0.0,0.0,0.500005,False,0.000000,1.391351e-110,7.630521e-110,3.216936e-107,-2.796922e-108,6.875543e-109
64,AAPL240816C00340000,340.0,0.00,0.00,0.0,0.0,0.500005,False,0.000000,1.677310e-126,9.849110e-126,4.152267e-123,-3.608031e-124,8.289105e-125
65,AAPL240816C00350000,350.0,0.00,0.00,0.0,0.0,0.500005,False,0.000000,5.333144e-143,3.332430e-142,1.404912e-139,-1.220165e-140,2.635713e-141
66,AAPL240816C00360000,360.0,0.00,0.00,0.0,0.0,0.500005,False,0.000000,5.150620e-160,3.406901e-159,1.436309e-156,-1.246897e-157,2.545611e-158
